<hr size='3'/>

In [ ]:
Pkg.add("PyPlot"); Pkg.add("Quandl"); Pkg.add("MarketData");

INFO: Installing LaTeXStrings v0.1.3
INFO: Installing PyCall v0.8.1
INFO: Installing PyPlot v1.5.3
INFO: Package database updated


In [ ]:
using PyPlot, Quandl, GARCH, MarketData

# Julia as an HPC alternative for Quant finance
<br/><br/>

![](images/banner.png)

<hr size='3'/>

## Breakdown of the talk
- Main features of Julia
<br/><br/>
- Derivative pricing
<br/><br/>
- Parallel processing
<br/><br/>
- Julia Community Groups
<br/><br/>
- High Performance Computing
<br/><br/>
- JuliaQuant
<br/><br/>
- Time series analysis
<br/><br/>
- Heteroskedasticity
<br/><br/>


<hr size='3'/>

## What make Julia different ?

* Julia is written in Julia.
<br/><br/>
* Uses LLVM / JIT compilation, so it is fast !!!
<br/><br/>
* Code is uncluttered, basic functions are built in.
<br/><br/>
* Homoiconic design: provides runtime macros.
<br/><br/>
* Easy to execute on multicore and parallel processors.
<br/><br/>
* Can connect with modules / libraries written in other languages
<br/><br/>
* Can spawn tasks and interact with other programs
<br/><br/>

<hr size='3'/>

![](images/bmark01.png)

<hr size='3'/>

In [1]:
using PyPlot
T = 100;
S0 = 100;
dt = 0.01;
v = 0.2;
r = 0.05;
q = 0.0;

LoadError: LoadError: ArgumentError: PyPlot not found in path
while loading In[1], in expression starting on line 1

In [ ]:
S = zeros(Float64,T)
S[1] = S0;
iseed = ccall( (:clock, "libc"), Int32, ());
srand(iseed);

dW = randn(T)*sqrt(dt);
[ S[t] = S[t-1]*(1 + (r - q - 0.5*v*v)*dt + v*dW[t] + 0.5*v*v*dW[t]*dW[t]) for t = 2:T ]

x = linspace(1,T);
plot(x,S)

In [ ]:
function asianOpt(N::Int64, T::Int64; S0=100.0, K=100.0, r=0.05, q=0.0, v=0.1, tma=0.25) 

# European Asian option.  
# Euler and Milstein discretization for Black-Scholes.

  @assert N > 0;
  @assert T > 0;
    
  dt = tma/T;
  S = zeros(Float64,T);
  A = zeros(Float64,N);

  for n = 1:N
    S[1] = S0 
    dW = randn(T)*sqrt(dt);
    for t = 2:T
      z0 = (r - q - 0.5*v*v)*S[t-1]*dt;
      z1 = v*S[t-1]*dW[t];
      z2 = 0.5*v*v*S[t-1]*dW[t]*dW[t];
      S[t] = S[t-1] + z0 + z1 + z2;
    end
    A[n] = mean(S);
  end

# Define the payoff and calculate price

  P = zeros(Float64,N);
  [ P[n] = max(A[n] - K, 0) for n = 1:N ];
  return exp(-r*tma)*mean(P);
    
end


In [ ]:
price = asianOpt(100000,100; K=102.0,v=0.2);
@printf "Option Price: %10.4f\n\n" price;


In [ ]:
@elapsed asianOpt(100000,100;K=102.0,v=0.2)

<hr size='3'/>

## Comparison with Python, R, Matlab etc.
![](./images/bmark02.png)

<hr size='3'/>

In [ ]:
# global CPU_CORES  (sysinfo.jl : line 22 )
#
ncores = ccall(:jl_cpu_cores, Int32, ())

In [ ]:
addprocs(3);

In [ ]:
n = nprocs()

In [ ]:
;cat Exotics.jl

In [ ]:
import Exotics
@time Exotics.asianOpt(1000000,100; K=102.0,v=0.2)

In [ ]:
@everywhere using Exotics

In [ ]:
tic();
@everywhere price = Exotics.asianOpt(250000,100; K=102.0,v=0.2);
totprice=0.0;
for i = 1:n
    totprice += remotecall_fetch(i,()->price)
end
totprice = totprice/n;
toc()

@printf "Asian price is %7.4f\n" totprice


<hr size='3'/>

## Julia Community Groups: 
- JuliaStats
- JuliaOpt
- JuliaWeb
<br/><br/>
- JuliaQuant
- JuliaFinMetriX
- QuantEcon
<br/><br/>
- JuliaParallel
- JuliaGPU
<br/><br/>

<hr size='3'/>

## JuliaStats group :
- StatsBase
- Distributions
- MultivariateStats
- KernelDensity / KernelEstimator
<br/><br/>
- Dataframes
- RDatasets
<br/><br/>
- GLM
- Lora / MCMC
- Clustering
<br/><br/>

<hr size='3'/>

## JuliaParallel group :
- ClusterManagers
- Elly, HDFS
- MPI
- Yeppp
<br/><br/>

In [ ]:
@time Exotics.asianOptYep(1000000,100; K=102.0,v=0.2)

<hr size='3'/>

## JuliaGPU group :
- CUDA, CUDArt, CUBLAS, CUFFT
- OpenCL, CLBLAS, CLFFT
<br/><br/>

![](images/GK110a.png)

In [ ]:
;cat OCL/saxpy.jl

![](images/OCL-timings-A.png)

In [ ]:
;cat OCL/vaxbyz.jl

In [ ]:
run(`./vaxbyz.sh`)

<hr size='3'/>


## JuliaQuant group :
- TimeSeries
- MarketData
- MarketTechnicals
<br/><br/>
- Quandl
- Ito
- GARCH
<br/><br/>
- FinancialAssets
- Grist
- TradeModels
- PortfolioModels
<br/><br/>

In [ ]:
## FinancialSeries.jl api demo
using MarketData, PyPlot

In [ ]:
#using FinancialSeries
include("./FS/FinancialSeries.jl")

In [ ]:
# example stock from MarketData
AAPL.colnames

In [ ]:
# construct financial time series
Apple = TimeArray(AAPL.timestamp, AAPL.values, AAPL.colnames, FinancialSeries.Stock(FinancialSeries.Ticker("AAPL")))

In [ ]:
ac = Apple["Close"].values;
tm = timestamp(Apple);
plot(tm,ac)

In [ ]:
plot(tm[end-500:end],ac[end-500:end])

In [ ]:
pc = percentchange(Apple["Close"],method="log").values;
tm = timestamp(Apple)[2:end];
plot(tm,pc)

<hr size='3'/>

## Generalized AutoRegressive Conditional Heteroskedasticity (GARCH) Process
<br/>
Developed in 1982 to describe an approach to estimate volatility in financial markets.
<br/><br/>
The GARCH process is often preferred in financial modeling because it gives a more real-world context than other forms when predicting the prices and rates of financial instruments. 
<br/><br/>
The general process for a GARCH model involves three steps:
- Estimate a best-fitting autoregressive model
- Compute autocorrelations of the error term
- Test for significance
<br/><br/>

In [ ]:
using Quandl
quotes = quandl("YAHOO/INDEX_GSPC", format="DataFrame")

In [ ]:
using PyPlot
qac = diff(log(array(quotes[:Adjusted_Close])))
plot(1:length(qac),qac)

In [ ]:
using GARCH
garchFit(qac)

<hr size='3'/>

## In Summary
- Coding in Julia is simple
<br/><br/>
- At has (even now) impressive functionality
<br/><br/>
- This can be extended by using C/Fortran/Python/R/Java modules and libraries
<br/><br/>
- Julia as implemented data frames, similar to those in R and Python/Pandas
<br/><br/>
- Community groups exist and are active in statistics, finance, econometrics etc.
<br/><br/>
- JuliaQuant already covers many of the principal financial requirements
<br/><br/>
